# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - należy zaimplementować w tym celu prostą wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # <<< HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [ ]:
!git clone https://github.com/lychanl/two-player-games.git

Cloning into 'two-player-games'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 106 (delta 45), reused 60 (delta 29), pack-reused 23
Receiving objects: 100% (106/106), 26.76 KiB | 2.06 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [ ]:
from typing import Tuple, List
from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove

import numpy as np
import math
import time
import random

Wielkość planszy

In [ ]:
ROW_COUNT = 6
COLUMN_COUNT = 7

In [ ]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour):
        self.game = game
        self.column_count = len(self.game.state.fields)
        self.row_count = len(self.game.state.fields[0])

    def evaluate_position(self, player: Player)->float:
        '''Calculate score (chance of winning) for player in current game state'''
        position_score = 0

        if self.game.is_finished():
          # Draw
          if self.game.get_winner() == None:
            position_score = 0
            return position_score
          # Player won
          elif player.char == self.game.get_winner().char:
              position_score = 10000
              return position_score
          # Player lost
          else:
              position_score = -10000
              return position_score

        #Non-terminal position
        position_score += self.evaluate_rows(player)
        position_score += self.evaluate_columns(player)
        position_score += self.evaluate_diagonals(player)
        return position_score


    def evaluate_rows(self, player: Player)->float:
        '''Calculate score in rows'''
        row_score = 0

        for row_index in range(self.row_count):
            # Creating a list with values of consecutive row
            row_values = [self.game.state.fields[column_index][row_index] for column_index in range(self.column_count)]
            # Evaluating every four in a row
            for column_index in range(self.column_count-3):
                four = row_values[column_index:column_index+3+1]
                row_score += self.evaluate_four(four, player)

        return row_score

    def evaluate_columns(self, player: Player)->float:
        '''Calculate score in columns'''
        row_score = 0

        for column_index in range(self.column_count):
          # Creating a list with values of consecutive column
            column_values = self.game.state.fields[column_index]
            # Evaluating every four in a column
            for row_index in range(self.row_count-3):
                four = column_values[row_index:row_index+3+1]
                row_score += self.evaluate_four(four, player)

        return row_score

    def evaluate_diagonals(self, player: Player)->float:
        '''Calculate score in diagonals'''
        # Finding all diagonals
        fields_array = np.array(self.game.state.fields)
        diagonals = [fields_array[::-1,:].diagonal(i) for i in range(-self.column_count + 1, self.row_count)]
        diagonals.extend(fields_array.diagonal(i) for i in range(self.row_count - 1, -self.column_count, -1))
        diagonals = [diag.tolist() for diag in diagonals]

        # Evaluating every four in every diagonal
        diagonal_score = 0
        for diagonal in diagonals:
            if len(diagonal) >= 4:
                for i in range(len(diagonal)-3):
                  diagonal_score += self.evaluate_four(diagonal[i:i+3+1], player)

        return diagonal_score


    def evaluate_four(self, fields: List, player: Player)->float:
        '''Grants a score for given four based on chance of winning. If no one can win, score = 0.'''

        player_fields = fields.count(player)
        empty_fields = fields.count(None)
        opponent_fields = 4 - fields.count(player) - fields.count(None)

        if player_fields == 3 and empty_fields == 1:
            return 10
        elif player_fields == 2 and empty_fields == 2:
            return 2
        elif opponent_fields == 3 and empty_fields == 1:
            return -10
        elif opponent_fields == 2 and empty_fields == 2:
            return -2
        else:
          return 0


    def get_best_move(self)->int:
        pass


    def is_valid_move(self, col_index:int)->bool:
        '''Checks if move in given column is possible in current game state'''
        valid_moves = self.game.get_moves()
        return ConnectFourMove(col_index) in valid_moves

    def minimax(self, depth, alpha:float, beta:float, is_maximizing_player:bool, uses_alpha_beta = True)-> Tuple[int, float]:

        if depth == 0 or self.game.is_finished():
            evaluated = self.evaluate_position(self.game.first_player)
            return (-1, evaluated)

        game_state_before_changes = self.game.state
        available_moves = self.game.get_moves()
        move_scores = []

        for move in available_moves:
            self.game.make_move(move)
            move_score = self.minimax(depth-1, alpha, beta, not is_maximizing_player, uses_alpha_beta)[1]
            move_scores.append(move_score)
            self.game.state = game_state_before_changes

            if is_maximizing_player:
                if uses_alpha_beta:
                    if move_score > alpha:
                        alpha = move_score
                        move_alpha = move.column
                    if move_score > beta:
                        return (move.column, move_score)
            else:
                if uses_alpha_beta:
                    if move_score < beta:
                        beta = move_score
                        move_beta = move.column
                    if move_score < alpha:
                        return (move.column, move_score)

        if is_maximizing_player:
            best_move_index = move_scores.index(max(move_scores))
        else:
            best_move_index = move_scores.index(min(move_scores))
        return (available_moves[best_move_index].column, move_scores[best_move_index])


    def play_predicted_moves(self, initial_depth, initial_is_maximizing_player):
        '''Calculates best move taking #initial_depth moves into account./n
           Plays the exact predicted moves, illustrates them on game board'''
        is_maximizing_player = initial_is_maximizing_player

        for depth in range(initial_depth, -1, -1):
            move, exp_score = self.minimax(depth, -math.inf, math.inf, is_maximizing_player, False)

            if move != -1:
                print("\nMove " + self.game.get_current_player().char + str(move))
                self.game.make_move(ConnectFourMove(move))
                print(self.game)
            else:
                print("Score: " + str(exp_score))

            is_maximizing_player = not is_maximizing_player


## Test solver vs solver

In [ ]:
def test_two_solvers(depth_p1, depth_p2, uses_alpha_beta = True, verbose = 0, existing_game = None):
    p1 = Player("a")
    p2 = Player("b")
    if existing_game is None:
        game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
    else:
        game = existing_game

    moves = []
    times = 0

    solver = MinMaxSolver(game)
    is_maximizing_player = True
    while not game.is_finished():
        if is_maximizing_player:
            depth = depth_p1
        else:
            depth = depth_p2
        start = time.time()
        best_move, score = solver.minimax(depth, -math.inf, math.inf, is_maximizing_player, uses_alpha_beta)
        end = time.time()
        times += (end-start)
        game.make_move(ConnectFourMove(best_move))
        moves.append(best_move)
        solver.game = game
        is_maximizing_player = not is_maximizing_player
        if verbose == 2:
            print(game)
            print("Score:", solver.minimax(0, -math.inf, math.inf, True, uses_alpha_beta)[1])

    if verbose == 1:
        print(game)
    if game.get_winner() == None:
        result = 0
    elif game.get_winner().char == p1.char:
        result = 1
    else:
        result = -1
    return (result, times)



####Equal depth

In [ ]:
test_two_solvers(1,1,True,1)

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[b][b][b][b][ ][ ][ ]
[b][a][a][a][ ][ ][ ]
[a][a][a][b][ ][ ][ ]


(-1, 0.05772113800048828)

In [ ]:
test_two_solvers(2,2,True,1)

Current player: b
[b][b][a][b][b][ ][b]
[b][a][b][a][a][a][a]
[b][b][a][b][b][b][a]
[a][a][b][b][a][a][b]
[b][b][a][a][b][a][a]
[a][a][b][a][a][b][a]


(1, 0.2475874423980713)

In [ ]:
test_two_solvers(3,3,True,1)

Current player: b
[a][ ][ ][ ][ ][ ][ ]
[b][b][ ][ ][ ][ ][ ]
[b][b][b][ ][ ][ ][ ]
[b][a][a][a][a][ ][ ]
[a][a][a][b][b][ ][ ]
[a][a][b][b][a][ ][ ]


(1, 0.8784451484680176)

In [ ]:
test_two_solvers(4,4,True,1)

Current player: a
[b][b][a][b][b][b][b]
[b][a][a][b][a][a][a]
[a][b][b][b][a][b][b]
[b][a][a][a][b][a][a]
[a][b][b][a][a][b][b]
[b][a][a][a][b][a][a]


(-1, 4.030450820922852)

In [ ]:
test_two_solvers(5,5,True,1)

Current player: a
[ ][ ][b][b][ ][ ][ ]
[a][ ][a][b][ ][ ][ ]
[b][ ][a][a][ ][b][ ]
[a][b][a][b][b][a][b]
[a][b][b][a][a][b][a]
[a][a][a][b][b][b][a]


(-1, 30.27606964111328)

####P1 greater depth

In [ ]:
test_two_solvers(2,1,True,1)

Current player: b
[b][a][ ][a][a][b][ ]
[b][b][ ][a][b][a][ ]
[b][a][ ][b][a][b][ ]
[a][b][ ][b][b][b][a]
[b][a][a][a][a][a][b]
[a][b][b][a][a][b][a]


(1, 0.17155718803405762)

In [ ]:
test_two_solvers(3,1,True,1)

Current player: a
[a][ ][ ][ ][ ][ ][ ]
[a][a][ ][ ][ ][ ][ ]
[b][b][a][ ][ ][ ][ ]
[b][b][b][b][ ][ ][ ]
[b][a][a][b][ ][ ][ ]
[a][a][a][b][ ][ ][ ]


(-1, 0.390411376953125)

In [ ]:
test_two_solvers(4,1,True,1)

Current player: b
[ ][ ][ ][b][a][ ][ ]
[ ][ ][b][a][a][ ][ ]
[a][ ][a][b][b][a][ ]
[b][ ][b][a][a][b][ ]
[b][ ][b][a][a][a][ ]
[b][ ][a][a][b][b][ ]


(1, 3.573117971420288)

In [ ]:
test_two_solvers(5,1,True,1)

Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ]
[b][a][b][b][ ][ ][ ]
[b][b][a][a][a][ ][ ]
[b][a][a][a][b][ ][ ]


(1, 9.675066709518433)

In [ ]:
test_two_solvers(3,2,True,1)

Current player: b
[b][a][ ][ ][ ][ ][ ]
[b][a][ ][ ][ ][ ][ ]
[a][b][b][a][a][ ][ ]
[a][b][a][a][b][ ][ ]
[b][a][b][b][a][ ][ ]
[a][a][b][a][b][b][ ]


(1, 0.9622964859008789)

In [ ]:
test_two_solvers(4,2,True,1)

Current player: b
[b][b][a][ ][ ][a][a]
[a][a][b][a][ ][b][b]
[b][b][a][b][ ][b][a]
[b][a][a][a][ ][a][b]
[a][a][b][b][ ][a][a]
[b][b][a][a][ ][b][b]


(1, 3.4718096256256104)

In [ ]:
test_two_solvers(4,3,True,1)

Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][b][a][ ][ ][ ]
[ ][b][a][b][ ][ ][ ]
[b][a][a][a][ ][ ][ ]
[a][b][b][a][ ][ ][ ]
[b][b][a][a][ ][ ][ ]


(1, 2.45713472366333)

In [ ]:
test_two_solvers(5,2,True,1)

Current player: a
[ ][b][a][b][a][a][ ]
[ ][b][a][b][b][b][ ]
[ ][a][b][b][a][a][b]
[ ][b][b][a][b][a][a]
[b][a][a][a][b][b][b]
[a][b][a][a][b][a][a]


(-1, 9.855266571044922)

In [ ]:
test_two_solvers(5,3,True,1)

Current player: a
[b][b][a][b][b][b][b]
[a][a][a][b][b][a][a]
[b][b][b][a][b][b][b]
[a][a][a][b][a][a][a]
[a][b][a][a][a][b][b]
[b][b][a][a][a][b][a]


(-1, 16.595146417617798)

In [ ]:
test_two_solvers(5,4,True,1)

Current player: b
[b][ ][b][a][ ][a][ ]
[a][ ][b][b][ ][a][ ]
[b][ ][b][b][a][a][ ]
[a][a][a][a][b][b][ ]
[b][b][b][a][a][a][ ]
[b][b][a][a][b][a][ ]


(1, 14.206987619400024)

#### MinMax VS MinMax alpha beta

In [ ]:
test_two_solvers(1,1,False,1)

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[b][b][b][b][ ][ ][ ]
[b][a][a][a][ ][ ][ ]
[a][a][a][b][ ][ ][ ]


(-1, 0.05211186408996582)

In [ ]:
test_two_solvers(1,1,True,1)

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[b][b][b][b][ ][ ][ ]
[b][a][a][a][ ][ ][ ]
[a][a][a][b][ ][ ][ ]


(-1, 0.03187727928161621)

In [ ]:
test_two_solvers(3,2,False,1)

Current player: b
[b][a][ ][ ][ ][ ][ ]
[b][a][ ][ ][ ][ ][ ]
[a][b][b][a][a][ ][ ]
[a][b][a][a][b][ ][ ]
[b][a][b][b][a][ ][ ]
[a][a][b][a][b][b][ ]


(1, 1.9072718620300293)

In [ ]:
test_two_solvers(3,2,True,1)

Current player: b
[b][a][ ][ ][ ][ ][ ]
[b][a][ ][ ][ ][ ][ ]
[a][b][b][a][a][ ][ ]
[a][b][a][a][b][ ][ ]
[b][a][b][b][a][ ][ ]
[a][a][b][a][b][b][ ]


(1, 0.6595687866210938)

In [ ]:
test_two_solvers(4,4,False,1)

Current player: a
[b][b][a][b][b][b][b]
[b][a][a][b][a][a][a]
[a][b][b][b][a][b][b]
[b][a][a][a][b][a][a]
[a][b][b][a][a][b][b]
[b][a][a][a][b][a][a]


(-1, 14.355254173278809)

In [ ]:
test_two_solvers(4,4,True,1)

Current player: a
[b][b][a][b][b][b][b]
[b][a][a][b][a][a][a]
[a][b][b][b][a][b][b]
[b][a][a][a][b][a][a]
[a][b][b][a][a][b][b]
[b][a][a][a][b][a][a]


(-1, 6.250838041305542)

#### Custom game

In [ ]:
p1 = Player("a")
p2 = Player("b")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
for move in [3, 0]:
    game.make_move(ConnectFourMove(move))

test_two_solvers(3,3,False, 1, game)

Current player: b
[b][ ][ ][ ][ ][ ][ ]
[a][ ][ ][b][ ][ ][ ]
[a][ ][ ][b][b][ ][ ]
[b][ ][a][a][a][a][ ]
[b][b][b][a][a][a][b]
[b][a][b][a][a][a][b]


(1, 1.8918402194976807)

## Test solver VS random moves

In [ ]:
def round_solver_random(depth_p1, verbose = 1, existing_game = None):
    p1 = Player("a")
    p2 = Player("b")
    if existing_game is None:
        game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
    else:
        game = existing_game

    moves = []
    solver = MinMaxSolver(game)
    is_maximizing_player = True

    while not game.is_finished():
        if is_maximizing_player:
            best_move, score = solver.minimax(depth_p1, -math.inf, math.inf, is_maximizing_player, True)
            move = best_move
        else:
            available_moves = game.get_moves()
            random_move = random.choice(available_moves).column
            move = random_move

        game.make_move(ConnectFourMove(move))
        moves.append(move)
        solver.game = game
        is_maximizing_player = not is_maximizing_player
        if verbose==2:
            print(game)
            print("Score:", solver.minimax(0, -math.inf, math.inf, True, True)[1])

    if verbose ==1:
        print(game)
    if game.get_winner() == None:
        result = 0
    elif game.get_winner().char == p1.char:
        result = 1
    else:
        result = -1
    return (result)

def test_solver_random(depth_p1, verbose = 0, rounds_nr=20, existing_game = None):
    if existing_game is not None:
        existing_game_roud_start = existing_game.state

    results_meaning = {1: "P1 won", 0: "Draw", -1: "P2 won"}
    results_count = {1: 0, 0: 0, -1: 0}
    for round_nr in range(rounds_nr):
        results_count[round_solver_random(depth_p1, verbose, existing_game)] += 1
        if existing_game is not None:
            existing_game.state = existing_game_roud_start

    for result in results_meaning.keys():
        print(results_meaning[result] +  " " + str(results_count[result]))


In [ ]:
test_solver_random(1, 0, 100)

P1 won 99
Draw 0
P2 won 1


##Wnioski:    
- wyniki przy zastosowaniu minmax z obinaniem alpha beta są takie same jak bez obcinania, a ich uzyskanie zajmuje mniej czasu
- zazwyczaj opłaca się używać większej głębokości przeszukiwania
- zastosowanie solvera prawie zawsze jest lepsze niż losowe ruchy

Shows predicted moves

In [ ]:
p1 = Player("a")
p2 = Player("b")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
game.make_move(ConnectFourMove(0))
game.make_move(ConnectFourMove(0))
game.make_move(ConnectFourMove(1))
game.make_move(ConnectFourMove(2))
print(game)
solver = MinMaxSolver(game)
print(solver.minimax(4,-math.inf, math.inf, True))
solver.play_predicted_moves(4, True)

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[a][a][b][ ][ ][ ][ ]
(1, -2)

Move a1
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[b][a][ ][ ][ ][ ][ ]
[a][a][b][ ][ ][ ][ ]

Move b0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[b][a][ ][ ][ ][ ][ ]
[a][a][b][ ][ ][ ][ ]

Move a0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[b][a][ ][ ][ ][ ][ ]
[a][a][b][ ][ ][ ][ ]

Move b1
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ]
[b][b][ ][ ][ ][ ][ ]
[b][a][ ][ ][ ][ ][ ]
[a][a][b][ ][ ][ ][ ]
Score: -2
